CD1_(/km^2)	0.335-0.999 mm

CD2_(/km^2)	1.00-4.75 mm

CD3_(/km^2)	4.75-200 mm

CD4_(/km^2) >200 mm

WD1_(g/km^2) 0.335-0.999 mm

WD2_(g/km^2) 1.00-4.75 mm

WD3_(g/km^2) 4.75-200 mm
		
WD4_(g/km^2) >200 mm			

In [1]:
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

Using TensorFlow backend.


**PREPROCESSING DATA**

In [0]:
import pandas as pd

from google.colab import drive

drive.mount('/content/gdrive/')
pollution_data = pd.read_csv('/content/gdrive/My Drive/AI_NoteBook/PROJECT/Import_File/PlasticMarinePollutionDataset.csv')

########## [PREPROCESSING] ##########
pollution_data.rename(columns = {'CD1_(/km^2)' : 'Count_Density_Class1Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD2_(/km^2)' : 'Count_Density_Class2Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD3_(/km^2)' : 'Count_Density_Class3Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD4 _(/km^2)' : 'Count_Density_Class4Plastic'}, inplace = True)

pollution_data.rename(columns = {'WD1_(g/km^2)' : 'Weight_Density_Class1Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD2_(g/km^2)' : 'Weight_Density_Class2Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD3_(g/km^2)' : 'Weight_Density_Class3Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD4_(g/km^2)' : 'Weight_Density_Class4Plastic'}, inplace = True)

pollution_data['Count_Density_Class1Plastic'] = pollution_data.Count_Density_Class1Plastic.str.replace(' ', '')
pollution_data['Count_Density_Class2Plastic'] = pollution_data.Count_Density_Class2Plastic.str.replace(' ', '')
pollution_data['Count_Density_Class3Plastic'] = pollution_data.Count_Density_Class3Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class2Plastic'] = pollution_data.Weight_Density_Class2Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class3Plastic'] = pollution_data.Weight_Density_Class3Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class4Plastic'] = pollution_data.Weight_Density_Class4Plastic.str.replace(' ', '')

pollution_data['Count_Density_Class1Plastic'] = pollution_data.Count_Density_Class1Plastic.str.replace(',', '').astype(float)
pollution_data['Count_Density_Class2Plastic'] = pollution_data.Count_Density_Class2Plastic.str.replace(',', '').astype(float)
pollution_data['Count_Density_Class3Plastic'] = pollution_data.Count_Density_Class3Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class2Plastic'] = pollution_data.Weight_Density_Class2Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class3Plastic'] = pollution_data.Weight_Density_Class3Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class4Plastic'] = pollution_data.Weight_Density_Class4Plastic.str.replace(',', '').astype(float)

pollution_data = pollution_data.drop_duplicates(keep = 'first')

pollution_data['Data'] = pollution_data.Date.astype(str)

pollution_data_4 = pollution_data[['Date','Count_Density_Class4Plastic']]

pollution_data_4 = pollution_data_4.dropna(how = 'any')

########## [REMOVE ZEROS] #########
pollution_data_4 = pollution_data_4.drop(index = pollution_data_4[pollution_data_4['Count_Density_Class4Plastic'] == 0].index)
###################################

pollution_data_4['Date'] = pd.to_datetime(pollution_data_4.Date)

pollution_data_4_GroupByDate = pollution_data_4.groupby('Date').Count_Density_Class4Plastic.sum()

pollution_data_4_GroupByDate = pd.DataFrame(pollution_data_4_GroupByDate)

pollution_data_4_GroupByDate ['Date'] = pollution_data_4_GroupByDate.index

######### [MOVING AVERAGES APPROACH FOR PLASTIC] ##########
pollution_data_4_GroupByDate ['Plastic_Moving_Average_9'] = pollution_data_4_GroupByDate.Count_Density_Class4Plastic.rolling(9, center=True).mean()
pollution_data_4_GroupByDate ['Plastic_Moving_Average_21'] = pollution_data_4_GroupByDate.Count_Density_Class4Plastic.rolling(21, center=True).mean()
pollution_data_4_GroupByDate ['Plastic_Moving_Average_36'] = pollution_data_4_GroupByDate.Count_Density_Class4Plastic.rolling(36, center=True).mean()
###########################################################

startdate = pd.to_datetime('2008/02/06')

pollution_data_4_GroupByDate ['Days after 2008-02-06'] = (pollution_data_4_GroupByDate.Date).subtract(startdate)

pollution_data_4_GroupByDate ['Days after 2008-02-06']  = pollution_data_4_GroupByDate['Days after 2008-02-06'].astype(str)

pollution_data_4_GroupByDate ['Days after 2008-02-06'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].str.replace(' days 00:00:00.000000000', '')
pollution_data_4_GroupByDate ['Days after 2008-02-06'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].astype(float)

######### [MOVING AVERAGES APPROACH FOR Days] ##########
pollution_data_4_GroupByDate ['Days_Moving_Average_9'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].rolling(9, center=True).mean()
pollution_data_4_GroupByDate ['Days_Moving_Average_21'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].rolling(21, center=True).mean()
pollution_data_4_GroupByDate ['Days_Moving_Average_36'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].rolling(36, center=True).mean()
###########################################################
print (pollution_data_4_GroupByDate.head(-5))

In [0]:
Dataset_1 = pollution_data_4_GroupByDate.filter(['Count_Density_Class4Plastic'])

Dataset_2 = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_9'])
Dataset_2 = Dataset_2.dropna(how='any')

Dataset_3 = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_21'])
Dataset_3 = Dataset_3.dropna(how='any')

Dataset_4 = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_36'])
Dataset_4 = Dataset_4.dropna(how='any')

Dataset_1 = Dataset_1.to_numpy()
Dataset_2 = Dataset_2.to_numpy()
Dataset_3 = Dataset_3.to_numpy()
Dataset_4 = Dataset_4.to_numpy()

training_data_length_1 = math.ceil (len(Dataset_1) * (0.8))
training_data_length_2 = math.ceil (len(Dataset_2) * (0.8))
training_data_length_3 = math.ceil (len(Dataset_3) * (0.8))
training_data_length_4 = math.ceil (len(Dataset_4) * (0.8))

print (Dataset_1)
print (training_data_length_1)
print (training_data_length_2)
print (training_data_length_3)
print (training_data_length_4)

In [0]:
scaler = MinMaxScaler(feature_range= (0,1))

scaled_data_1 = scaler.fit_transform(Dataset_1)
scaled_data_2 = scaler.fit_transform(Dataset_2)
scaled_data_3 = scaler.fit_transform(Dataset_3)
scaled_data_4 = scaler.fit_transform(Dataset_4)

print (scaled_data_1)
print (scaled_data_2)
print (scaled_data_3)
print (scaled_data_4)

In [0]:
train_data_1 = scaled_data_1[0:training_data_length_1, :]
train_data_2 = scaled_data_1[0:training_data_length_2, :]
train_data_3 = scaled_data_1[0:training_data_length_3, :]
train_data_4 = scaled_data_1[0:training_data_length_4, :]

X_train_1 = []
y_train_1 = []

X_train_2 = []
y_train_2 = []

X_train_3 = []
y_train_3 = []

X_train_4 = []
y_train_4 = []

for i in range (30, len (train_data_1)) :
  X_train_1.append(train_data_1[i-30:i, 0])
  y_train_1.append(train_data_1[i, 0])

for i in range (30, len (train_data_2)) :
  X_train_2.append(train_data_2[i-30:i, 0])
  y_train_2.append(train_data_2[i, 0])

for i in range (30, len (train_data_3)) :
  X_train_3.append(train_data_3[i-30:i, 0])
  y_train_3.append(train_data_3[i, 0])

for i in range (30, len (train_data_4)) :
  X_train_4.append(train_data_4[i-30:i, 0])
  y_train_4.append(train_data_4[i, 0])

X_train_1, y_train_1 = np.array(X_train_1), np.array(y_train_1)
X_train_2, y_train_2 = np.array(X_train_2), np.array(y_train_2)  
X_train_3, y_train_3 = np.array(X_train_3), np.array(y_train_3)  
X_train_4, y_train_4 = np.array(X_train_4), np.array(y_train_4)     

In [0]:
X_train_1 = np.reshape(X_train_1 ,(X_train_1.shape[0], X_train_1.shape[1], 1))
X_train_2 = np.reshape(X_train_2, (X_train_2.shape[0], X_train_2.shape[1], 1))
X_train_3 = np.reshape(X_train_3, (X_train_3.shape[0], X_train_3.shape[1], 1))
X_train_4 = np.reshape(X_train_4, (X_train_4.shape[0], X_train_4.shape[1], 1))
print (X_train_1.shape)
print (X_train_2.shape)
print (X_train_3.shape)
print (X_train_4.shape)

In [0]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (X_train_1.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train_1, y_train_1, batch_size = 1, epochs = 3)

test_data_1 = scaled_data_1[training_data_length_1 - 30:, :]

X_test_1 = []
y_test_1 = Dataset_1[training_data_length_1:, :]

for i in range(30, len(test_data_1)) :
  X_test_1.append(test_data_1[i-30: i, 0])

X_test_1 = np.array(X_test_1)

X_test_1 = np.reshape(X_test_1, (X_test_1.shape[0], X_test_1.shape[1], 1))

Predictions = model.predict(X_test_1)
Predictions = scaler.inverse_transform(Predictions)

In [0]:
RMSE = np.sqrt(np.mean(Predictions - y_test_1)**2)

RMSE

In [0]:
train = pollution_data_4_GroupByDate.filter(['Count_Density_Class4Plastic'])[:training_data_length_1]
valid = pollution_data_4_GroupByDate.filter(['Count_Density_Class4Plastic'])[training_data_length_1:]
valid ['Predictions'] = Predictions

plt.figure(figsize=(16,8))
plt.title('Count Density', fontsize = 30)
plt.xlabel('Date', fontsize = 15, color = 'W')
plt.ylabel('Count Density per Unit Area', fontsize = 15, color = 'W')
plt.plot(train['Count_Density_Class4Plastic'], color = 'darkgray')
plt.plot(valid['Count_Density_Class4Plastic'], color = 'skyblue')
plt.plot(valid['Predictions'], color = 'blue')
plt.legend(['Train', 'Actual', 'Predictions'], loc = 'upper right')
plt.show()

In [0]:
valid

In [0]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (X_train_2.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train_2, y_train_2, batch_size = 1, epochs = 3)

test_data_2 = scaled_data_2[training_data_length_2 - 30:, :]

X_test_2 = []
y_test_2 = Dataset_2[training_data_length_2:, :]

for i in range(30, len(test_data_2)) :
  X_test_2.append(test_data_2[i-30: i, 0])

X_test_2 = np.array(X_test_2)

X_test_2 = np.reshape(X_test_2, (X_test_2.shape[0], X_test_2.shape[1], 1))

Predictions = model.predict(X_test_2)
Predictions = scaler.inverse_transform(Predictions)

RMSE = np.sqrt(np.mean(Predictions - y_test_2)**2)

RMSE

In [0]:
train = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_9']).dropna(how='any')[:training_data_length_2]
valid = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_9']).dropna(how='any')[training_data_length_2:]
valid ['Predictions'] = Predictions

plt.figure(figsize=(16,8))
plt.title('Count Density', fontsize = 30)
plt.xlabel('Date', fontsize = 15, color = 'W')
plt.ylabel('Count Density per Unit Area', fontsize = 15, color = 'W')
plt.plot(train['Plastic_Moving_Average_9'], color = 'darkgray')
plt.plot(valid['Plastic_Moving_Average_9'], color = 'skyblue')
plt.plot(valid['Predictions'], color = 'blue')
plt.legend(['Train', 'Actual', 'Predictions'], loc = 'upper right')
plt.show()

In [0]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (X_train_3.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train_3, y_train_3, batch_size = 3, epochs = 3)

test_data_3 = scaled_data_3[training_data_length_3 - 30:, :]

X_test_3 = []
y_test_3 = Dataset_3[training_data_length_3:, :]

for i in range(30, len(test_data_3)) :
  X_test_3.append(test_data_3[i-30: i, 0])

X_test_3 = np.array(X_test_3)

X_test_3 = np.reshape(X_test_3, (X_test_3.shape[0], X_test_3.shape[1], 1))

Predictions = model.predict(X_test_3)
Predictions = scaler.inverse_transform(Predictions)

RMSE = np.sqrt(np.mean(Predictions - y_test_3)**2)

RMSE

In [0]:
train = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_21']).dropna(how='any')[:training_data_length_3]
valid = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_21']).dropna(how='any')[training_data_length_3:]
valid ['Predictions'] = Predictions

plt.figure(figsize=(16,8))
plt.title('Count Density', fontsize = 30)
plt.xlabel('Date', fontsize = 15, color = 'W')
plt.ylabel('Count Density per Unit Area', fontsize = 15, color = 'W')
plt.plot(train['Plastic_Moving_Average_21'], color = 'darkgray')
plt.plot(valid['Plastic_Moving_Average_21'], color = 'skyblue')
plt.plot(valid['Predictions'], color = 'blue')
plt.legend(['Train', 'Actual', 'Predictions'], loc = 'upper right')
plt.show()

In [0]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (X_train_4.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train_4, y_train_4, batch_size = 3, epochs = 3)

test_data_4 = scaled_data_4[training_data_length_4 - 30:, :]

X_test_4 = []
y_test_4 = Dataset_4[training_data_length_4:, :]

for i in range(30, len(test_data_4)) :
  X_test_4.append(test_data_4[i-30: i, 0])

X_test_4 = np.array(X_test_4)

X_test_4 = np.reshape(X_test_4, (X_test_4.shape[0], X_test_4.shape[1], 1))

Predictions = model.predict(X_test_4)
Predictions = scaler.inverse_transform(Predictions)

RMSE = np.sqrt(np.mean(Predictions - y_test_4)**2)

RMSE

In [0]:
train = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_36']).dropna(how='any')[:training_data_length_4]
valid = pollution_data_4_GroupByDate.filter(['Plastic_Moving_Average_36']).dropna(how='any')[training_data_length_4:]
valid ['Predictions'] = Predictions

plt.figure(figsize=(16,8))
plt.title('Count Density', fontsize = 30)
plt.xlabel('Date', fontsize = 15, color = 'W')
plt.ylabel('Count Density per Unit Area', fontsize = 15, color = 'W')
plt.plot(train['Plastic_Moving_Average_36'], color = 'darkgray')
plt.plot(valid['Plastic_Moving_Average_36'], color = 'skyblue')
plt.plot(valid['Predictions'], color = 'blue')
plt.legend(['Train', 'Actual', 'Predictions'], loc = 'upper right')
plt.show()